<a href="https://colab.research.google.com/github/atik666/AugmmentationCode/blob/TransferLearning/ensembleClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow import keras
from keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
X_train1 = np.load("/content/drive/MyDrive/Colab/Main/X_train1.npy")
y_train1 = np.load("/content/drive/MyDrive/Colab/Main/y_train1.npy")
X_val = np.load("/content/drive/MyDrive/Colab/Main/X_val.npy")
y_val = np.load("/content/drive/MyDrive/Colab/Main/y_val.npy")
X_test = np.load("/content/drive/MyDrive/Colab/Main/X_test.npy")
y_test = np.load("/content/drive/MyDrive/Colab/Main/y_test.npy")

X_aug = np.load("/content/drive/MyDrive/Colab/Aug/X_aug1.npy")
y_aug = np.load("/content/drive/MyDrive/Colab/Aug/y_aug1.npy")

In [ ]:
X_train = np.concatenate((X_train1, X_aug))
y_train = np.concatenate((y_train1, y_aug))
print('Number of images in X_train', X_train.shape[0])
print('Number of images in X_val', X_val.shape[0])
print('Number of images in X_test', X_test.shape[0])

In [ ]:
IMAGE_SHAPE = (224, 224, 3)
      
model_name = "inception_v3"
feature_extractor_model="https://tfhub.dev/google/tf2-preview/%s/feature_vector/4" %model_name

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=IMAGE_SHAPE, trainable=False)
       
num_class = 10

In [ ]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_class)
])

model.summary()          
         
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
     
# Train your model using the early stopping callback
h_callback = model.fit(X_train, y_train, batch_size = 32,
           epochs = 50, validation_data = (X_val, y_val))

loss, acc= model.evaluate(X_test, y_test)
print('Test Accuracy: %f' % (acc*100))

def plot_loss(loss,val_loss):
  plt.figure()
  plt.plot(loss)
  plt.plot(val_loss)
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper right')
  plt.show()
  
def plot_accuracy(acc,val_acc):
  # Plot training & validation accuracy values
  plt.figure()
  plt.plot(acc)
  plt.plot(val_acc)
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='lower right')
  plt.show()
  
# Plot train vs test loss during training
plot_loss(h_callback.history['loss'], h_callback.history['val_loss'])

# Plot train vs test accuracy during training
plot_accuracy(h_callback.history['accuracy'], h_callback.history['val_accuracy'])

from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
conf = confusion_matrix(y_test, y_pred)
print(conf)
print('acc : ', np.trace(conf)/np.sum(conf)*100)